In [1]:
import os
import pydicom
import numpy as np
import torch 
import glob
import sys
import time 

import matplotlib.pyplot as plt
import matplotlib.path as mplPath

import pylab as pl

from threading import Timer

from collections import Counter
from collections import defaultdict

## Load data into project

In [2]:
class Project(object): 
    """
    Loads a project, appends new projects and makes subject 
    data accessable. Most variables are a list with each instance
    in the list representing a subject.

    subjects: list with subject names 
    subprojects: list with project each subject belongs to
    masks: list with masks as 3d np arrays (H x W x # slices)
    masks_shape: list with shape of each mask
    pcmras: list with masks as 3d np arrays
    pcmras_shape: list with shape of each pcmra
    """
    def __init__(self, project, root="/scratch/ptenkaate/Data"): 
        # create paths for masks and pcmras
        self.root = root # constant
        self.project = project # variable
        
        self.m_path = os.path.join(self.root, self.project, "mask")
        self.p_path = os.path.join(self.root, self.project, "pcmra")
        
        
        # retrieve all file names 
        if not os.path.exists(self.m_path):
            raise(Exception("This project does not exist, please specify a different project"))
        
        for root, folders, mask_files in os.walk(self.m_path):
            break   
        for root, folders, pcmra_files in os.walk(self.p_path):
            break   
        
        # only keep subjects that have mask and pcmra 
        self.subjects = sorted([file for file in mask_files if file in pcmra_files])   
        
        # list with project name for every subject, 
        # needed if multiple projects are added
        self.subprojects = [project for i in range(len(self.subjects))]
        
        # print files that did not have a mask
#         no_pcmra = [file for file in mask_files if file not in pcmra_files]
#         print("Masks without PCMRA:", no_pcmra)

        # retrieve 4 lists, two containing np arrays, # and two containing shapes in tuple form
        self.masks, self.masks_shape = self.get_files_in_folder(self.m_path, self.subjects)
        self.pcmras, self.pcmras_shape = self.get_files_in_folder(self.p_path, self.subjects)
        
        # check whether the mask and pcmra shape are same
        for subject, mask_shape, pcmra_shape in zip(self.subjects, self.masks_shape, self.pcmras_shape):
            if mask_shape != pcmra_shape: 
                print(f"Shapes of subject {subject} are not equal:")
                print(f"Mask shape: {mask_shape}, \t PCMRA shape: {pcmra_shape}")
        
        
    def get_files_in_folder(self, path, subjects): 
        """
        Append all files in a folder to a list. 
        files: list with np arrays of shape (h x w x # slices)
        files_shape: list with all tuples stated above
        """
        files = []
        files_shape = []

        for subj in subjects: 
            with open(os.path.join(path, subj), "rb") as f:
                file = np.load(f)
                files.append(file)
                files_shape.append(file.shape)

        return files, files_shape
    
    
    def append_project(self, project):
        """
        Used if a new project from another folder needs to be appended
        to the current project. 
        """
        # create a new project
        new_project = Project(project, self.root)
        
        # append the new project to list with current projects
        if type(self.project) == str: 
            self.project = [self.project, project]
        else: 
            self.project.append(project)
        
        # add all new subject data to existing project 
        self.subjects += new_project.subjects
        self.subprojects += new_project.subprojects
        self.masks += new_project.masks
        self.masks_shape += new_project.masks_shape
        self.pcmras += new_project.pcmras
        self.pcmras_shape += new_project.pcmras_shape
                
    
    def filter_dimension_shape(self, dim, size, print_dropped=False): 
        """
        All subjects that do not satisfy to the given 
        size of a dimension are removed. 
        dim: dimension that needs to be compared 
        size: prefered size of the dimension. 
        """
        
        # new lists to add subjects that satisfy
        subjects, subprojects, masks, masks_shape, pcmras, pcmras_shape = [], [], [], [], [], []
        
        for i in range(len(self.subjects)): 
            if self.masks_shape[i][dim] == size and self.pcmras_shape[i][dim] == size: 
                subjects.append(self.subjects[i])
                subprojects.append(self.subprojects[i])
                masks.append(self.masks[i])
                masks_shape.append(self.masks_shape[i])
                pcmras.append(self.pcmras[i])
                pcmras_shape.append(self.pcmras_shape[i])
            else: 
                
                if print_dropped:
                    print(f"Subject nr {i}, name {self.subjects[i]} does not have a size {size} at dimension {dim}")
        
        self.subjects = subjects
        self.subprojects = subprojects
        self.masks = masks
        self.masks_shape = masks_shape
        self.pcmras = pcmras
        self.pcmras_shape = pcmras_shape       
    
    
    def get_subject(self, i): 
        """
        Returns subject name, pcmra array and masks array.
        """
        return self.subjects[i], self.pcmras[i], self.masks[i]
     
        
    def get_number_of_subjects(self):
        """
        Returns the number of subjects in the project.
        """
        return len(self.masks)

    
    def get_number_of_slices(self):
        """
        Returns the total number of slices of all subjects combined.
        """
        i = 0
        for mask in self.masks: 
            i += mask.shape[2]

        return i
    
    def normalize(self, minimum=0, maximum=1): 
        """
        Normalizes masks and pcmras between -1 and 1 based 
        on their maximum and minimum values.
        """
        self.masks = [(maximum - minimum)*((mask - mask.min()) / (mask.max() - mask.min())) + minimum
                      for mask in self.masks]
        
        self.pcmras = [(maximum - minimum)*((pcmra - pcmra.min()) / (pcmra.max() - pcmra.min())) + minimum
                       for pcmra in self.pcmras]

In [3]:
# project = Project("Aorta CoA")

# for i in range(project.get_number_of_subjects()):
#     subject, mask, pcmra = project.get_subject(i) 
#     print(subject, pcmra.shape, mask.shape)
#     break

# for pcmra in project.pcmras: 
#     print(pcmra.shape)
#     print(pcmra.flatten().shape)
#     print("0% \t", np.percentile(pcmra, 0))
#     print("1% \t", np.percentile(pcmra, 1))
#     print("5% \t", np.percentile(pcmra, 5))
#     print("50% \t", np.percentile(pcmra, 50))
#     print("95% \t", np.percentile(pcmra, 95))
#     print("99% \t", np.percentile(pcmra, 99))
#     print("100% \t", np.percentile(pcmra, 100))
    


## Show an image interactively

In [4]:
import sys
import numpy
    
    
class Show_images(object):
    """
    Scroll through slices. Takes an unspecified number of subfigures per figure.
    suptitles: either a str or a list. Represents the 
    main title of a figure. 
    images_titles: a list with tuples, each tuple an np.array and a 
    title for the array subfigure. 
    """
    def __init__(self, suptitles, *images_titles):
        # if string if given, make list with that title for 
        # each slice.
        if type(suptitles) == str: 
            self.suptitles = []
            for i in range(images_titles[0][0].shape[2]): 
                self.suptitles.append(suptitles)
        else: 
            self.suptitles = suptitles
                    
        self.fig, self.ax = plt.subplots(1,len(images_titles))

        # split tuples with (image, title) into lists
        self.images = [x[0] for x in images_titles]
        self.titles = [x[1] for x in images_titles]

        # get the number of slices that are to be shown
        rows, cols, self.slices = self.images[0].shape        
        self.ind = 0

        self.fig.suptitle(self.suptitles[self.ind]) # set title 

        self.plots = []
        
        # start at slice 10 if more than 20 slices, 
        # otherwise start at middle slice.
        if self.images[0].shape[2] > 20: 
            self.ind = 10
        else:
            self.ind = self.images[0].shape[2] // 2
        
        # make sure ax is an np array
        if type(self.ax) == np.ndarray:
            pass
        else: 
            self.ax = np.array([self.ax])
        
        # create title for each subfigure in slice
        for (sub_ax, image, title) in zip(self.ax, self.images, self.titles): 
            sub_ax.set_title(title)
            plot = sub_ax.imshow(image[:, :, self.ind])
            self.plots.append(plot)

            
        # link figure to mouse scroll movement
        self.plot_show = self.fig.canvas.mpl_connect('scroll_event', self.onscroll)
        
        # show window maximized 
#         figManager = plt.get_current_fig_manager()
#         figManager.window.showMaximized()
        
#         plt.ion()
#         plt.show()

    def onscroll(self, event):
        """
        Shows next or previous slice with mouse scroll.
        """
        if event.button == 'up':
            self.ind = (self.ind - 1) % self.slices
        else:
            self.ind = (self.ind + 1) % self.slices
        
        self.update()
        

    def update(self):
        """
        Updates the figure.
        """
        self.fig.suptitle(self.suptitles[self.ind])
        
        for plot, image in zip(self.plots, self.images):
            plot.set_data(image[:, :, self.ind])
        
        self.ax[0].set_ylabel('Slice Number: %s' % self.ind)
        self.plots[0].axes.figure.canvas.draw()

In [5]:
print("Imported classes.")

Imported classes.
